In [1]:
from astropy.table import Table
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [2]:
gz_data = Table.read('../../fits/full_sample_debiased_w_low_z.fits')
print('Loaded GZ galaxies!')

# Load the matched catalogues:

nsa = Table.read('matched_fits/nsa_matched.fits')

chang_input = Table.read('matched_fits/chang_input_matched.fits')
chang_output = Table.read('matched_fits/chang_output_matched.fits')

Loaded GZ galaxies!


In [ ]:
os.chdir('../../astro_codes/')
import volume_limiting, mass_limiting, make_dictionaries, bar_plots
questions = make_dictionaries.questions
os.chdir('../Gas-SF/SF/')

In [ ]:
Mag_limit = volume_limiting.get_sample_limits(0.055,17)
mass_limit = mass_limiting.get_mass_limit(gz_data,z_limit=0.055,mag_limit=17,mass_equation=mass_limiting.baldry_equation
                                          ,low_z_limit=0.02,column_names=['REDSHIFT_1', 'PETROMAG_MR'],
                                          colours=['PETROMAG_MU','PETROMAG_MR'])

in_volume = ((gz_data['REDSHIFT_1'] >= 0.02) &
             (gz_data['PETROMAG_MR'] <= Mag_limit) &
             (gz_data['REDSHIFT_1'] <= 0.055))

in_mass = gz_data['LOGMSTAR_BALDRY06'] >= mass_limit
samples = Table()
samples['in_volume'] = in_volume
samples['in_mass'] = (in_volume) & (in_mass)

In [ ]:
p_spiral = (gz_data['t01_smooth_or_features_a02_features_or_disk_debiased_rh']*
            gz_data['t02_edgeon_a05_no_debiased_rh']*
            gz_data['t04_spiral_a08_spiral_debiased_rh'])

N_spiral = (gz_data['t04_spiral_a08_spiral_count'] - 
            gz_data['t11_arms_number_a37_cant_tell_count'])

p_ok = p_spiral > 0.5
N_ok = N_spiral >= 5
in_spiral = (p_ok) & (N_ok)
samples['in_spiral'] = in_spiral

question = 't11_arms_number'
answers = questions[question]['answers'][:-1]
answer_array = np.array([gz_data[question + '_' + A + '_debiased_rh'].data for A in answers]).T
m = np.argmax(answer_array,axis=1)
samples['m'] = m

In [ ]:
in_sample = (samples['in_spiral']) & (samples['in_mass'])

# WISE detections?

In [ ]:
flags = Table()
flags['has_wise_detection'] = (chang_output['flag_w'] == 1) #&
                                #(chang_output['flag_chi2'] == 1))
flags['good_w3'] = chang_output['flag_w3'] == 1
flags['good_w4'] = chang_output['flag_w4'] == 1

In [ ]:
arm_labels = ['1','2','3','4','5+']
wise_detections = Table()
totals = []
for m in range(5):
    ok_detection = []
    ok_detection.append(np.sum(flags[(in_sample) & (samples['m'] == m)]['has_wise_detection']))
    ok_detection.append(np.sum(flags[(in_sample) & (samples['m'] == m)]['good_w3']))
    ok_detection.append(np.sum(flags[(in_sample) & (samples['m'] == m)]['good_w4']))
    wise_detections[arm_labels[m]] = ok_detection   
    totals.append(np.sum(samples[in_sample]['m'] == m))

In [ ]:
_ = bar_plots.comparison_bar_chart(wise_detections,['Unique \n detecton','W3','W4'],['b','r','y'],
                                   normalise='percent',linewidth=1.5,ylabel=r'$\%$',column_totals=totals)

# FUV/NUV detections?

In [ ]:
flags['has_uv_detection'] = nsa['mask'] == True
flags['good_fuv'] = (nsa['AMIVAR'][:,0] > 0) & (nsa['DFLAGS'][:,0] == 0)
flags['good_nuv'] = (nsa['AMIVAR'][:,1] > 0) & (nsa['DFLAGS'][:,1] == 0)

In [ ]:
uv_detections = Table()
totals = []
for m in range(5):
    ok_detection = []
    ok_detection.append(np.sum(flags[(in_sample) & (samples['m'] == m)]['has_uv_detection']))
    ok_detection.append(np.sum(flags[(in_sample) & (samples['m'] == m)]['good_fuv']))
    ok_detection.append(np.sum(flags[(in_sample) & (samples['m'] == m)]['good_nuv']))
    uv_detections[arm_labels[m]] = ok_detection   
    totals.append(np.sum(samples[in_sample]['m'] == m))

In [ ]:
_ = bar_plots.comparison_bar_chart(uv_detections,['Unique \n detecton','FUV','NUV'],['b','r','y'],
                                   normalise='percent',linewidth=1.5,ylabel=r'$\%$',column_totals=totals)

# Compare FUV, r + W3 bands directly?

In [ ]:
os.chdir('../../astro_codes/')
import imp
imp.reload(kde_plotting)
from magnitude_conversions import FUV_Mag_to_luminosity
questions = make_dictionaries.questions
os.chdir('../Gas-SF/SF/')

In [ ]:
from sklearn.metrics import matthews_corrcoef

mcc = matthews_corrcoef(flags['has_wise_detection'],flags['good_fuv'])
print('Matthews correlation coefficient = {} (random?)'.format(np.round(mcc,decimals=2)))

In [ ]:
def Mag_to_lum(Mag,M_sun):
    return 10**((M_sun-Mag)/2.5)

nsa['fuv_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,0],16.02))
nsa['nuv_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,1],10.18))
nsa['u_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,2],6.75))
nsa['g_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,3],5.33))
nsa['r_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,4],4.67))
nsa['i_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,5],4.48))
nsa['z_lum'] = np.log10(Mag_to_lum(nsa['ABSMAG'][:,6],4.42))

xyz_array = np.array([nsa['fuv_lum'],nsa['r_lum'],chang_output['lrest_w3']]).T
xyz_mask = [(nsa['AMIVAR'][:,0] > 0) & (nsa['DFLAGS'][:,0] == 0),
            (nsa['AMIVAR'][:,4] > 0) & (nsa['DFLAGS'][:,4] == 0),
            (chang_output['flag_w'] == 1) & (chang_output['flag_w3'] == 1)]

xyz_mask = [i[in_sample] for i in xyz_mask]

#xyz_mask = Table(xyz_mask)
xyz_array = xyz_array[in_sample]
xyz = Table(xyz_array,names=('$L_{FUV}/L_{\odot}$','$L_{r}/L_{\odot}$','$L_{W3}/L_{\odot}$'))

In [ ]:
def FUV_Mag_to_luminosity(Mag):
    '''
    Function for converting from absolute magnitude -> luminosity.
    
    Inputs:
    -------
    Mag: absolute magnitude
    
    Outputs:
    --------
    L: luminosity (in ergs/s/Hz)
    '''
    
    K = 4*math.pi*(10*u.pc)**2 #4pi(D**2)
    S_Jy = 10**((23.9-Mag)/2.5)*10**(-6)*u.Jy # SDSS 'Pogson' magnitude
    L = K*S_Jy
    return L.to(u.erg/u.s/u.Hz)

In [ ]:
def Mag_to_lum(Mag):
    S = 3631*10**(Mag/-2.5)*u.Jy # AB -> flux density
    L = S*(4*math.pi)*(10*u.pc)**2 # absolute magnitude = 10pc
    return L.to(u.erg/u.s/u.Hz)


def lum_to_sfr(lum):
    return np.log10(1.08e-28*L.value) # conversion factor (Salim+2007) 


def modify_magnitude(FUV,NUV,r):    
    A_FUV = np.zeros(len(FUV))
    red = NUV - r < 4
    blue = red == False
    A_FUV[red] = red_convert(FUV[red],NUV[red])
    A_FUV[blue] = blue_convert(FUV[blue],NUV[blue])
    return A_FUV

    
def red_convert(FUV,NUV):
    A_FUV = np.zeros(len(FUV))
    red =  FUV - NUV < 0.95
    blue = red == False
    A_FUV[red] = 3.32*(FUV[red]-NUV[red]) + 0.22
    A_FUV[blue] = 3.37
    return A_FUV


def blue_convert(FUV,NUV):
    A_FUV = np.zeros(len(FUV))
    red =  FUV - NUV < 0.9
    blue = red == False
    A_FUV[red] = 2.99*(FUV[red]-NUV[red]) + 0.27
    A_FUV[blue] = 2.96
    return A_FUV

In [ ]:
FUV, NUV, r = [nsa[in_sample]['ABSMAG'][:,i] for i in [0,1,4]]
mask = ((FUV != -999) &
        (NUV != -999) &
        (r != -999))
FUV, NUV, r = [FUV[mask],NUV[mask],r[mask]]

A_FUV = modify_magnitude(FUV,NUV,r)
FUV_corrected = FUV - A_FUV

In [ ]:
L = Mag_to_lum(FUV_corrected)
_ = plt.hist(np.log10(L.value),bins=np.linspace(25,30,100))

In [ ]:
sfr = lum_to_sfr(L)

In [ ]:
_ = plt.hist(sfr,bins=np.linspace(-2,2,100))